In [2]:
import numpy as np
import numpy
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
import scipy.io
mat = scipy.io.loadmat('DadosLista4.mat')

In [4]:
dados = np.array(mat['Dados'])
y = np.array(mat['y'])

In [5]:
sizeDados = len(dados)
dados1, dados2, dados3 = dados[ : int((sizeDados/3))] , dados[int((sizeDados/3)) : int((2*sizeDados/3)) ], dados[ int((2*sizeDados/3)) : ]
y1, y2, y3 = y[ : int((sizeDados/3))] , y[int((sizeDados/3)) : int((2*sizeDados/3)) ], y[ int((2*sizeDados/3)) : ]

In [6]:
#Classe 1 treino
randomTreino1 = np.random.choice(dados1.shape[0], 200, replace=False)
dadosTreino1 = dados1[randomTreino1]
yTreino1 = y1[randomTreino1]

#Classe 1 treino
randomTreino2 = np.random.choice(dados2.shape[0], 200, replace=False)
dadosTreino2 = dados2[randomTreino2]
yTreino2 = y2[randomTreino2]

#Classe 1 treino
randomTreino3 = np.random.choice(dados3.shape[0], 200, replace=False)
dadosTreino3 = dados3[randomTreino3]
yTreino3 = y3[randomTreino3]



In [7]:
#Classe 1 teste
a1_rows = set(map(tuple, dados1))
a2_rows = set(map(tuple, dadosTreino1))
dadosTeste1 = np.array(list(a1_rows.difference(a2_rows)))
yTeste1 = np.ones((300,1))

#Classe 2 teste
a1_rows = set(map(tuple, dados2))
a2_rows = set(map(tuple, dadosTreino2))
dadosTeste2 = np.array(list(a1_rows.difference(a2_rows)))
yTeste2 = np.full((300,1), 2)

#Classe 3 teste
a1_rows = set(map(tuple, dados3))
a2_rows = set(map(tuple, dadosTreino3))
dadosTeste3 = np.array(list(a1_rows.difference(a2_rows)))
yTeste3 = np.full( (300,1), 3)


In [8]:
dadosTreino = np.concatenate((dadosTreino1, dadosTreino2, dadosTreino3), axis=0)
len(dadosTreino)

600

In [9]:
dadosTeste = np.concatenate((dadosTeste1, dadosTeste2, dadosTeste3), axis=0)
len(dadosTeste)

900

In [10]:
yTreino = np.concatenate((yTreino1, yTreino2, yTreino3), axis=0 )
len(yTreino)

600

In [11]:
yTeste = np.concatenate( (yTeste1, yTeste2, yTeste3), axis=0)
len(yTeste)

900

In [22]:
#calcula mi
miGeral = dadosTreino.mean(axis=0)
mi1 = dadosTreino1.mean(axis=0)
mi2 = dadosTreino2.mean(axis=0)
mi3 = dadosTreino3.mean(axis=0)

In [38]:
#somatorio
n1 = dadosTreino1.shape[0]
n2 = dadosTreino2.shape[0]
n3 = dadosTreino3.shape[0]
nGeral = dadosTreino.shape[0]

aux1 = np.reshape( (dadosTreino1[0] - mi1), (2,1))
aux1T = np.transpose(np.reshape( (dadosTreino1[0] - mi1), (2,1)))
sum1 = [ aux1.dot(aux1T) for linha in range(n1)]

sum1 = np.sum(np.array(sum1), axis=0)

ec1 = sum1 / (n1 - 1)

aux2 = np.reshape( (dadosTreino2[0] - mi2), (2,1))
aux2T = np.transpose(np.reshape( (dadosTreino2[0] - mi2), (2,1)))
sum2 = [ aux2.dot(aux2T) for linha in range(n2)]

sum2 = np.sum(np.array(sum2), axis=0)

ec2 = np.array(sum2) / (n2 - 1)

aux3 = np.reshape( (dadosTreino3[0] - mi3), (2,1))
aux3T = np.transpose(np.reshape( (dadosTreino3[0] - mi3), (2,1)))
sum3 = [ aux3.dot(aux3T) for linha in range(n3)]

sum3 = np.sum(np.array(sum3), axis=0)

ec3 = np.array(sum3) / (n3 - 1)

auxGeral = np.reshape( (dadosTreino[0] - miGeral), (2,1))
auxGeralT = np.transpose(np.reshape( (dadosTreino[0] - miGeral), (2,1)))
sumGeral = [ auxGeral.dot(auxGeralT) for linha in range(nGeral)]

sumGeral = np.sum(np.array(sumGeral), axis=0)

ecGeral = np.array(sumGeral) / (nGeral - 1)

In [59]:
#Probabilidade de x para a classe C referente ao mi e ec(sigma) => P(x|c)
def prob(mi, ec, x):
    
    # (x - miC)T
    matrixT = np.transpose(np.reshape( (x - mi), (2,1)))
    print(ec)
    print(np.linalg.inv(ec))
    # (x -miC)T * SigmaC
    aux = matrixT.dot(np.linalg.inv(ec))
    # (x -miC)T * SigmaC * (x- miC)
    final = aux.dot(np.reshape((x - mi), (2,1)))
    # exp( -1/2 *  (x -miC)T * SigmaC * (x- miC) )
    result = np.exp( (-1/2) * final)
    print(result)
    return result

In [57]:
#Funcao que define qual das 3 probabilidades foi a maior
def maxY(y1, y2, y3):
    if y1 >= y2 and y1 >= y3:
        return 1
    elif y2 >= y3 :
        return 2
    else:
        return 3

In [60]:
yResults = []
for dadoLinha in dadosTeste :
    y1 = prob(mi1, ec1, dadoLinha)
    y2 = prob(mi2, ec2, dadoLinha)
    y3 = prob(mi3, ec3, dadoLinha)
    yResults.append(maxY(y1,y2,y3))
    

&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.50574979  0.24971486]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.50574979  0.24971486]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.50574979  0.24971486]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.63809838  0.19560684]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15

&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.87368314  0.57878707]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.92248764  0.64258068]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.92248764  0.64258068]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.92248764  0.64258068]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17

[ 0.83796962  0.65717633]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.83796962  0.65717633]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.83796962  0.65717633]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.8270305   0.82641892]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01

[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.76488292  0.90204384]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.76488292  0.90204384]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.40023759  0.2918518 ]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.40023759  0.2918518 ]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e

 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.48574337  0.44363742]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.48574337  0.44363742]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.48574337  0.44363742]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.66635832  0.41276352]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.

...

[ 0.13421211  1.09287918]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.13421211  1.09287918]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.13421211  1.09287918]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.13967161  0.97310618]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01

[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.34255856  1.04128214]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.34255856  1.04128214]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.10203876  1.15092888]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.10203876  1.15092888]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.

[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.2320984  0.9835451]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.30336559  0.89359958]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.30336559  0.89359958]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.30336559  0.89359958]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.163827

&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.23861824  1.06413692]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.23861824  1.06413692]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.17384595  1.10520864]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.17384595  1.10520864]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16

&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.13408725  0.95093774]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16]
 [  1.07127078e+16   9.76301582e+15]]
[[ 0.]]
&&&&&&&
[ 0.1886083   1.00051326]
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[ 0.13408725  0.95093774]
=-------
[[ 0.0106511  -0.00624016]
 [-0.00624016  0.00365592]]
[[  6.81851521e+16   1.16382762e+17]
 [  1.16382762e+17   1.98649515e+17]]
[[ 0.]]
&&&&&&&
[ 0.70820735  0.50695192]
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[ 0.25069387  1.04032339]
=-------
[[ 0.05511553  0.03567277]
 [ 0.03567277  0.02308871]]
[[  2.16580138e+15  -3.34622998e+15]
 [ -3.34622998e+15   5.17002859e+15]]
[[ 0.]]
&&&&&&&
[ 0.60374101  0.60178027]
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[ 0.25069387  1.04032339]
=-------
[[ 0.01059076 -0.01162097]
 [-0.01162097  0.0127514 ]]
[[  1.17547805e+16   1.07127078e+16

In [ ]:
print("&&&&&&&")
    print(mi)
    print(ec)
    print(x)
    print("=-------")